In [1]:
###### utils
import os
import random
import numpy as np
import pickle

import torch
from torch.autograd import Variable

__author__ = "Yifeng Tao"


def bool_ext(rbool):
    """ Solve the problem that raw bool type is always True.
    Parameters
    ----------
    rbool: str
      should be True of False.
    """

    if rbool not in ["True", "False"]:
        raise ValueError("Not a valid boolean string")

    return rbool == "True"


def load_dataset(input_dir="data", deg_shuffle=False):
    """ Load dataset, modify, and shuffle`.
    Parameters
    ----------
    input_dir: str
      input directory of dataset
    deg_shuffle: bool
      whether to shuffle DEG or not
    Returns
    -------
    dataset: dict
      dict of lists, including SGAs, cancer types, DEGs, patient barcodes
    """

    # load dataset
    data = pickle.load( open(os.path.join(input_dir, "dataset_324.pkl"), "rb") )
    can_r = data["can"] # cancer type index of tumors: list of int
    sga_r = data["sga"] # SGA index of tumors: list of list
    deg = data["deg"]   # DEG binary matrix of tumors: 2D array of 0/1
    tmr = data["tmr"]   # barcodes of tumors: list of str

    # shift the index of cancer type by +1, 0 is for padding
    can = np.asarray([[x+1] for x in can_r], dtype=int)

    # shift the index of SGAs by +1, 0 is for padding
    num_max_sga = max([len(s) for s in sga_r])
    sga = np.zeros( (len(sga_r), num_max_sga), dtype=int )
    for idx, line in enumerate(sga_r):
        line = [s+1 for s in line]
        sga[idx,0:len(line)] = line

    # shuffle DEGs
    if deg_shuffle:
        rng = list(range(deg.shape[1]))
        for idx in range(deg.shape[0]):
            random.shuffle(rng)
            deg[idx] = deg[idx][rng]

    # shuffle whole dataset
    rng = list(range(len(can)))
    random.Random(2019).shuffle(rng)
    can = can[rng]
    sga = sga[rng]
    deg = deg[rng]
    tmr = [tmr[idx] for idx in rng]

    dataset = {"can":can, "sga":sga, "deg":deg, "tmr":tmr}

    return dataset


def split_dataset(dataset, ratio=0.66):
    """ Split the dataset according to the ratio of training/test sets.
    Parameters
    ----------
    dataset: dict
        dict of lists, including SGAs, cancer types, DEGs, patient barcodes
    ratio: float
        size(train_set)/size(train_set+test_set)
    Returns
    -------
    train_set, test_set: dict
    """

    num_sample = len(dataset["can"])
    num_train_sample = int(num_sample*ratio)

    train_set = {"sga":dataset["sga"][:num_train_sample],
               "can":dataset["can"][:num_train_sample],
               "deg":dataset["deg"][:num_train_sample],
               "tmr":dataset["tmr"][:num_train_sample]}
    test_set = {"sga":dataset["sga"][num_train_sample:],
              "can":dataset["can"][num_train_sample:],
              "deg":dataset["deg"][num_train_sample:],
              "tmr":dataset["tmr"][num_train_sample:]}

    return train_set, test_set


def wrap_dataset(sga, can, deg, tmr):
    """ Wrap default numpy or list data into PyTorch variables.
    """

    dataset = {"sga": Variable(torch.LongTensor(sga)),
             "can": Variable(torch.LongTensor(can)),
             "deg": Variable(torch.FloatTensor(deg)),
             "tmr": tmr}

    return dataset


def get_minibatch(dataset, index, batch_size, batch_type="train"):
    """ Get a mini-batch dataset for training or test.
    Parameters
    ----------
    dataset: dict
        dict of lists, including SGAs, cancer types, DEGs, patient barcodes
    index: int
        starting index of current mini-batch
    batch_size: int
    batch_type: str
        batch strategy is slightly different for training and test
        "train": will return to beginning of the queue when `index` out of range
        "test": will not return to beginning of the queue when `index` out of range
    Returns
    -------
    batch_dataset: dict
        a mini-batch of the input `dataset`.
    """

    sga = dataset["sga"]
    can = dataset["can"]
    deg = dataset["deg"]
    tmr = dataset["tmr"]

    if batch_type == "train":
        batch_sga = [ sga[idx%len(sga)]
            for idx in range(index,index+batch_size) ]
        batch_can = [ can[idx%len(can)]
            for idx in range(index,index+batch_size) ]
        batch_deg = [ deg[idx%len(deg)]
            for idx in range(index,index+batch_size) ]
        batch_tmr = [ tmr[idx%len(tmr)]
            for idx in range(index,index+batch_size) ]
    elif batch_type == "test":
        batch_sga = sga[index:index+batch_size]
        batch_can = can[index:index+batch_size]
        batch_deg = deg[index:index+batch_size]
        batch_tmr = tmr[index:index+batch_size]

    batch_dataset = wrap_dataset(
        batch_sga,
        batch_can,
        batch_deg,
        batch_tmr)

    return batch_dataset


def evaluate(labels, preds, epsilon=1e-4):
    """ Calculate performance metrics given ground truths and prediction results.
    Parameters
    ----------
    labels: matrix of 0/1
        ground truth labels
    preds: matrix of float in [0,1]
        predicted labels
    epsilon: float
        a small Laplacian smoothing term to avoid zero denominator
    Returns
    -------
    precision: float
    recall: float
    f1score: float
    accuracy: float
    """

    flat_labels = np.reshape(labels,-1)
    flat_preds = np.reshape(np.around(preds),-1)

    accuracy = np.mean(flat_labels == flat_preds)
    true_pos = np.dot(flat_labels, flat_preds)
    precision = 1.0*true_pos/flat_preds.sum()
    recall = 1.0*true_pos/flat_labels.sum()

    f1score = 2*precision*recall/(precision+recall+epsilon)

    return precision, recall, f1score, accuracy

In [2]:
data = pickle.load( open(os.path.join('', "dataset_324.pkl"), "rb") )
can_r = data["can"] # cancer type index of tumors: list of int
sga_r = data["sga"] # SGA index of tumors: list of list
deg = data["deg"]   # DEG binary matrix of tumors: 2D array of 0/1
tmr = data["tmr"]   # barcodes of tumors: list of str

In [3]:
""" Base model of GIT and its variants.
"""
import torch
import torch.nn as nn

__author__ = "Yifeng Tao"


class ModelBase(nn.Module):
    """ Base models for all models.
    """

    def __init__(self, args):
        """ Initialize the hyperparameters of model.
        Parameters
        ----------
        args: arguments for initializing the model.
        """

        super(ModelBase, self).__init__()

        self.epsilon = 1e-4

        self.input_dir = args.input_dir
        self.output_dir = args.output_dir

        self.sga_size = args.sga_size
        self.deg_size = args.deg_size
        self.can_size = args.can_size

        self.num_max_sga = args.num_max_sga

        self.embedding_size = args.embedding_size
        self.hidden_size = args.hidden_size
        self.attention_size = args.attention_size
        self.attention_head = args.attention_head

        self.learning_rate = args.learning_rate
        self.dropout_rate = args.dropout_rate
        self.weight_decay = args.weight_decay

        self.initializtion = args.initializtion
        self.attention = args.attention
        self.cancer_type = args.cancer_type
        self.deg_shuffle = args.deg_shuffle


    def build(self):
        """ Define modules of the model.
        """
        raise NotImplementedError


    def forward(self):
        """ Define the data flow across modules of the model.
        """
        raise NotImplementedError


    def train(self):
        """ Train the model using training set.
        """
        raise NotImplementedError


    def test(self):
        """ Test the model using test set.
        """
        raise NotImplementedError


    def load_model(self, path="data/trained_model.pth"):
        """ Load trained parameters of the model.
        """
        print("Loading model from "+path)
        self.load_state_dict(torch.load(path))


    def save_model(self, path="data/trained_model.pth"):
        """ Save learnable parameters of the trained model.
        """
        print("Saving model to "+path)
        torch.save(self.state_dict(), path)

In [4]:
""" Implementation of GIT model and its variants.
"""
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

__author__ = "Yifeng Tao"


class GIT(ModelBase):
    """ GIT model and its variants.
    """

    def __init__(self, args, **kwargs):
        """ Initialize the model.
        Parameters
        ----------
        args: arguments for initializing the model.
        """
        super(GIT, self).__init__(args, **kwargs)


    def build(self):
        """ Define modules of the model.
        """

        self.layer_sga_emb = nn.Embedding(
            num_embeddings=self.sga_size+1,
            embedding_dim=self.embedding_size,
            padding_idx=0)

        self.layer_can_emb = nn.Embedding(
            num_embeddings=self.can_size+1,
            embedding_dim=self.embedding_size,
            padding_idx=0)

        self.layer_w_0 = nn.Linear(
            in_features=self.embedding_size,
            out_features=self.attention_size,
            bias=True)

        self.layer_beta = nn.Linear(
            in_features=self.attention_size,
            out_features=self.attention_head,
            bias=True)

        self.layer_dropout_1 = nn.Dropout(p=self.dropout_rate)

        self.layer_w_1 = nn.Linear(
            in_features=self.embedding_size,
            out_features=self.hidden_size,
            bias=True)

        self.layer_dropout_2 = nn.Dropout(p=self.dropout_rate)

        self.layer_w_2 = nn.Linear(
            in_features=self.hidden_size,
            out_features=self.deg_size,
            bias=True)

        if self.initializtion:
            gene_emb_pretrain = np.load(os.path.join("", "gene_emb_pretrain_324.npy"))
            self.layer_sga_emb.weight.data.copy_(torch.from_numpy(gene_emb_pretrain))

        self.optimizer = optim.Adam(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay)


    def forward(self, sga_index, can_index):
        """ Forward process.
        Parameters
        ----------
        sga_index: list of SGA index vectors.
        can_index: list of cancer type indices.
        Returns
        -------
        preds: 2D array of float in [0, 1]
          predicted gene expression
        hid_tmr: 2D array of float
          hidden layer of MLP
        emb_tmr: 2D array of float
          tumor embedding
        emb_sga: 2D array of float
          stratified tumor embedding
        attn_wt: 2D array of float
          attention weights of SGAs
        """

        # cancer type embedding
        emb_can = self.layer_can_emb(can_index)
        emb_can = emb_can.view(-1,self.embedding_size)

        # gene embedings
        E_t = self.layer_sga_emb(sga_index)

        # squeeze and tanh-curve the gene embeddings
        E_t_flatten = E_t.view(-1, self.embedding_size)
        E_t1_flatten = torch.tanh( self.layer_w_0(E_t_flatten) )

        # multiplied by attention heads
        E_t2_flatten = self.layer_beta(E_t1_flatten)
        E_t2 = E_t2_flatten.view(-1, self.num_max_sga, self.attention_head)

        # normalize by softmax
        E_t2 = E_t2.permute(1,0,2)
        A = F.softmax(E_t2)
        A = A.permute(1,0,2)

        if self.attention:
            # multi-head attention weighted sga embedding:
            emb_sga = torch.sum( torch.bmm( A.permute(0,2,1), E_t ), dim=1)
            emb_sga = emb_sga.view(-1,self.embedding_size)
        else:
            # if not using attention, simply sum up SGA embeddings
            emb_sga = torch.sum(E_t, dim=1)
            emb_sga = emb_sga.view(-1, self.embedding_size)

        # if use cancer type input, add cancer type embedding
        if self.cancer_type:
            emb_tmr = emb_can+emb_sga
        else:
            emb_tmr = emb_sga

        # MLP decoder
        emb_tmr_relu = self.layer_dropout_1(F.relu(emb_tmr))
        hid_tmr = self.layer_w_1(emb_tmr_relu)
        hid_tmr_relu = self.layer_dropout_2(F.relu(hid_tmr))
        preds = F.sigmoid(self.layer_w_2(hid_tmr_relu))

        # attention weights
        attn_wt = torch.sum(A, dim=2)
        attn_wt = attn_wt.view(-1, self.num_max_sga)

        return preds, hid_tmr, emb_tmr, emb_sga, attn_wt


    def train(self, train_set, test_set,
            batch_size=None, test_batch_size=None,
            max_iter=None, max_fscore=None,
            test_inc_size=None, **kwargs):
        """ Train the model until max_iter or max_fscore reached.
        Parameters
        ----------
        train_set: dict
          dict of lists, including SGAs, cancer types, DEGs, patient barcodes
        test_set: dict
        batch_size: int
        test_batch_size: int
        max_iter: int
          max number of iterations that the training will run
        max_fscore: float
          max test F1 score that the model will continue to train itself
        test_inc_size: int
          interval of running a test/evaluation
        """

        for iter_train in range(0, max_iter+1, batch_size):
            batch_set = get_minibatch(train_set, iter_train, batch_size,batch_type="train")
            preds, _, _, _, _ = self.forward(batch_set["sga"], batch_set["can"])
            labels = batch_set["deg"]

            self.optimizer.zero_grad()
            loss = -torch.log( self.epsilon +1-torch.abs(preds-labels) ).mean()
            loss.backward()
            self.optimizer.step()

            if test_inc_size and (iter_train % test_inc_size == 0):
                labels, preds, _, _, _, _, _ = self.test(test_set, test_batch_size)
                precision, recall, f1score, accuracy = evaluate(
                    labels, preds, epsilon=self.epsilon)
                print("[%d,%d], f1_score: %.3f, acc: %.3f"% (iter_train//len(train_set["can"]),
                      iter_train%len(train_set["can"]), f1score, accuracy))

            if f1score >= max_fscore:
                break

        # self.save_model(os.path.join('', "trained_model.pth"))


    def test(self, test_set, test_batch_size, **kwargs):
        """ Run forward process over the given whole test set.
        Parameters
        ----------
        test_set: dict
          dict of lists, including SGAs, cancer types, DEGs, patient barcodes
        test_batch_size: int
        Returns
        -------
        labels: 2D array of 0/1
          groud truth of gene expression
        preds: 2D array of float in [0, 1]
          predicted gene expression
        hid_tmr: 2D array of float
          hidden layer of MLP
        emb_tmr: 2D array of float
          tumor embedding
        emb_sga: 2D array of float
          stratified tumor embedding
        attn_wt: 2D array of float
          attention weights of SGAs
        tmr: list of str
          barcodes of patients/tumors
        """

        labels, preds, hid_tmr, emb_tmr, emb_sga, attn_wt, tmr = [], [], [], [], [], [], []

        for iter_test in range(0, len(test_set["can"]), test_batch_size):
            batch_set = get_minibatch(test_set, iter_test, test_batch_size, batch_type="test")
            batch_preds, batch_hid_tmr, batch_emb_tmr, batch_emb_sga, batch_attn_wt = self.forward(
              batch_set["sga"], batch_set["can"])
            batch_labels = batch_set["deg"]

            labels.append(batch_labels.data.numpy())
            preds.append(batch_preds.data.numpy())
            hid_tmr.append(batch_hid_tmr.data.numpy())
            emb_tmr.append(batch_emb_tmr.data.numpy())
            emb_sga.append(batch_emb_sga.data.numpy())
            attn_wt.append(batch_attn_wt.data.numpy())
            tmr = tmr + batch_set["tmr"]

        labels = np.concatenate(labels,axis=0)
        preds = np.concatenate(preds,axis=0)
        hid_tmr = np.concatenate(hid_tmr,axis=0)
        emb_tmr = np.concatenate(emb_tmr,axis=0)
        emb_sga = np.concatenate(emb_sga,axis=0)
        attn_wt = np.concatenate(attn_wt,axis=0)
        
        return labels, preds, hid_tmr, emb_tmr, emb_sga, attn_wt, tmr

In [6]:
dataset = load_dataset(input_dir='', deg_shuffle=False)
train_set, test_set = split_dataset(dataset, ratio=0.66)

In [53]:
""" Demo of training and evaluating GIT model and its variants.
"""
import os
import argparse

__author__ = "Yifeng Tao"


parser = argparse.ArgumentParser()

parser.add_argument('-f')

parser.add_argument("--train_model", help="whether to train model or load model", type=bool_ext, default=True)

parser.add_argument("--input_dir", help="directory of input files", type=str, default="data")
parser.add_argument("--output_dir", help="directory of output files", type=str, default="data")

parser.add_argument("--embedding_size", help="embedding dimension of genes and tumors", type=int, default=512)
parser.add_argument("--hidden_size", help="hidden layer dimension of MLP decoder", type=int, default=1024)
parser.add_argument("--attention_size", help="size of attention parameter beta_j", type=int, default=400)
parser.add_argument("--attention_head", help="number of attention heads", type=int, default=128)

parser.add_argument("--learning_rate", help="learning rate for Adam", type=float, default=1e-4)
parser.add_argument("--max_iter", help="maximum number of training iterations", type=int, default=3072*20)
parser.add_argument("--max_fscore", help="Max F1 score to early stop model from training", type=float, default=0.7)
parser.add_argument("--batch_size", help="training batch size", type=int, default=16)
parser.add_argument("--test_batch_size", help="test batch size", type=int, default=512)
parser.add_argument("--test_inc_size", help="increment interval size between log outputs", type=int, default=256)
parser.add_argument("--dropout_rate", help="dropout rate", type=float, default=0.5)
parser.add_argument("--weight_decay", help="coefficient of l2 regularizer", type=float, default=1e-5)

# GIT variants:
# args.initializtion=False -> GIT-init
parser.add_argument("--initializtion", help="whether to use pre-trained gene embeddings or not", type=bool_ext, default=True)
# args.attention=False -> GIT-attn
parser.add_argument("--attention", help="whether to use attention mechanism or not", type=bool_ext, default=True)
# args.cancer_type=False -> GIT-can
parser.add_argument("--cancer_type", help="whether to use cancer type or not", type=bool_ext, default=True)
# args.deg_shuffle=False -> DEG-shuffled
parser.add_argument("--deg_shuffle", help="whether to shuffle DEGs or not", type=bool_ext, default=False)

args = parser.parse_args()

# Above default hyper parameters tuned in full GIT model;
# Ablated GIT variants have slight different tuned hyper parameters:

# if args.cancer_type == False:
#  args.max_iter = 3072*40
# elif args.attention == False:
#  args.max_iter = 3072*40
#  args.learning_rate = 0.0003

In [56]:
print("Loading dataset...")
dataset = load_dataset(input_dir='', deg_shuffle=args.deg_shuffle)
train_set, test_set = split_dataset(dataset, ratio=0.66)

args.can_size = dataset["can"].max()        # cancer type dimension
args.sga_size = dataset["sga"].max()        # SGA dimension
args.deg_size = dataset["deg"].shape[1]     # DEG output dimension
args.num_max_sga = dataset["sga"].shape[1]  # maximum number of SGAs in a tumor

Loading dataset...


In [46]:
### Hyperparameters set

params_lst = ["hidden_size", "attention_size", "attention_head", "learning_rate", "max_iter", "max_fscore", "batch_size",
             "test_batch_size", "test_inc_size", "dropout_rate", "weight_decay", "initializtion", "attention",
             "cancer_type", "deg_shuffle"]
params_value_dict = {"hidden_size" : [256, 512], "attention_size":[200, 400], 
                     "attention_head": [128, 256], "learning_rate": [1e-4, 2e-4, 3e-4], 
                     "max_iter": [3072*20, 3072*40], "max_fscore": [0.7, 0.8], 
                     "batch_size": [16], "test_batch_size": [512], 
                     "test_inc_size": [256], "dropout_rate": [0.5], 
                     "weight_decay": [1e-5], "initializtion": [False], 
                     "attention": [True], "cancer_type": [True], "deg_shuffle": [False]}
import itertools as it

allparams = params_value_dict
combinations = it.product(*(params_value_dict[param] for param in allparams))
combinations_lst = list(combinations)
print(len(combinations_lst))

hyper_param_dict = {}
for i in range(len(combinations_lst)):
    hyper_param_dict[i] = {}
    for j in range(len(params_lst)):
        hyper_param_dict[i][params_lst[j]] = combinations_lst[i][j]

96


In [54]:
import numpy
print("Original Hyperparameters:")
print(args)

Original Hyperparameters:
Namespace(f='C:\\Users\\Shi Yingfei\\AppData\\Roaming\\jupyter\\runtime\\kernel-639b48bb-9bbd-41e6-83b7-070f3f769436.json', train_model=True, input_dir='data', output_dir='data', embedding_size=512, hidden_size=1024, attention_size=400, attention_head=128, learning_rate=0.0001, max_iter=61440, max_fscore=0.7, batch_size=16, test_batch_size=512, test_inc_size=256, dropout_rate=0.5, weight_decay=1e-05, initializtion=True, attention=True, cancer_type=True, deg_shuffle=False)


In [83]:
model_eval_dict = {}
for i in hyper_param_dict:
    params_dict = hyper_param_dict[i]
    args.hidden_size = params_dict["hidden_size"]
    args.attention_size=params_dict["attention_size"] 
    args.attention_head=params_dict["attention_head"] 
    args.learning_rate=params_dict["learning_rate"] 
    args.max_iter=params_dict["max_iter"]
    args.max_fscore=params_dict["max_fscore"] 
    args.batch_size=params_dict["batch_size"] 
    args.test_batch_size=params_dict["test_batch_size"]
    args.test_inc_size=params_dict["test_inc_size"]
    args.dropout_rate=params_dict["dropout_rate"]
    args.weight_decay=params_dict["weight_decay"] 
    args.initializtion=params_dict["initializtion"] 
    args.attention=params_dict["attention"]
    args.cancer_type=params_dict["cancer_type"] 
    args.deg_shuffle=params_dict["deg_shuffle"]
    model = GIT(args)   # initialize GIT model (with no pretrained gene embeddings)
    model.build()                               # build GIT model

    if args.train_model:                        # train from scratch
        print(f"Model_{i}_Training...")
        model.train(train_set, test_set,
            batch_size=args.batch_size,
            test_batch_size=args.test_batch_size,
            max_iter=args.max_iter,
            max_fscore=args.max_fscore,
            test_inc_size=args.test_inc_size)
        model.save_model(os.path.join(f"ht_models/init_trained_model_{i}.pth"))
    else:                                      # or directly load trained model
        model.load_model(os.path.join(f"ht_models/init_trained_model_{i}.pth"))
        
    # evaluation
    print("Evaluating...")
    labels, preds, _, _, _, _, _ = model.test(test_set, test_batch_size=args.test_batch_size)
    precision, recall, f1score, accuracy = evaluate(labels, preds, epsilon=model.epsilon)
    print("prec=%.3f, recall=%.3f, F1=%.3f, acc=%.3f"%(precision, recall, f1score, accuracy))
    
    model_eval_dict[i] = {"precision":precision, "recall":recall, "f1score":f1score, "accuracy":accuracy}

Model_0_Training...


C:\Users\Shi Yingfei\AppData\Local\Temp\ipykernel_15320\910214608.py:112: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  A = F.softmax(E_t2)


[0,0], f1_score: 0.385, acc: 0.502
[0,256], f1_score: 0.384, acc: 0.522
[0,512], f1_score: 0.383, acc: 0.532
[0,768], f1_score: 0.382, acc: 0.537
[0,1024], f1_score: 0.383, acc: 0.541
[0,1280], f1_score: 0.383, acc: 0.542
[0,1536], f1_score: 0.382, acc: 0.542
[0,1792], f1_score: 0.382, acc: 0.543
[0,2048], f1_score: 0.383, acc: 0.544
[0,2304], f1_score: 0.383, acc: 0.545
[0,2560], f1_score: 0.382, acc: 0.546
[0,2816], f1_score: 0.383, acc: 0.547
[1,195], f1_score: 0.383, acc: 0.549
[1,451], f1_score: 0.382, acc: 0.549
[1,707], f1_score: 0.382, acc: 0.551
[1,963], f1_score: 0.382, acc: 0.553
[1,1219], f1_score: 0.382, acc: 0.554
[1,1475], f1_score: 0.382, acc: 0.556
[1,1731], f1_score: 0.382, acc: 0.559
[1,1987], f1_score: 0.382, acc: 0.560
[1,2243], f1_score: 0.381, acc: 0.562
[1,2499], f1_score: 0.381, acc: 0.564
[1,2755], f1_score: 0.381, acc: 0.566
[2,134], f1_score: 0.381, acc: 0.569
[2,390], f1_score: 0.380, acc: 0.571
[2,646], f1_score: 0.380, acc: 0.574
[2,902], f1_score: 0.379,

[19,633], f1_score: 0.482, acc: 0.750
[19,889], f1_score: 0.482, acc: 0.750
[19,1145], f1_score: 0.482, acc: 0.751
[19,1401], f1_score: 0.490, acc: 0.751
[19,1657], f1_score: 0.488, acc: 0.751
[19,1913], f1_score: 0.490, acc: 0.752
[19,2169], f1_score: 0.489, acc: 0.752
[19,2425], f1_score: 0.490, acc: 0.752
[19,2681], f1_score: 0.496, acc: 0.753
[20,60], f1_score: 0.497, acc: 0.753
[20,316], f1_score: 0.498, acc: 0.753
[20,572], f1_score: 0.499, acc: 0.754
[20,828], f1_score: 0.500, acc: 0.755
[20,1084], f1_score: 0.500, acc: 0.755
[20,1340], f1_score: 0.500, acc: 0.755
[20,1596], f1_score: 0.503, acc: 0.755
[20,1852], f1_score: 0.505, acc: 0.755
[20,2108], f1_score: 0.504, acc: 0.755
[20,2364], f1_score: 0.507, acc: 0.756
[20,2620], f1_score: 0.513, acc: 0.757
[20,2876], f1_score: 0.513, acc: 0.757
[21,255], f1_score: 0.513, acc: 0.757
[21,511], f1_score: 0.517, acc: 0.757
[21,767], f1_score: 0.518, acc: 0.758
[21,1023], f1_score: 0.514, acc: 0.758
Saving model to ht_models/init_trai

[16,1840], f1_score: 0.429, acc: 0.735
[16,2096], f1_score: 0.427, acc: 0.736
[16,2352], f1_score: 0.428, acc: 0.734
[16,2608], f1_score: 0.431, acc: 0.736
[16,2864], f1_score: 0.438, acc: 0.737
[17,243], f1_score: 0.434, acc: 0.737
[17,499], f1_score: 0.437, acc: 0.737
[17,755], f1_score: 0.438, acc: 0.738
[17,1011], f1_score: 0.435, acc: 0.738
[17,1267], f1_score: 0.439, acc: 0.739
[17,1523], f1_score: 0.442, acc: 0.738
[17,1779], f1_score: 0.445, acc: 0.738
[17,2035], f1_score: 0.449, acc: 0.739
[17,2291], f1_score: 0.447, acc: 0.739
[17,2547], f1_score: 0.453, acc: 0.741
[17,2803], f1_score: 0.452, acc: 0.741
[18,182], f1_score: 0.455, acc: 0.742
[18,438], f1_score: 0.452, acc: 0.741
[18,694], f1_score: 0.452, acc: 0.742
[18,950], f1_score: 0.450, acc: 0.742
[18,1206], f1_score: 0.456, acc: 0.743
[18,1462], f1_score: 0.454, acc: 0.742
[18,1718], f1_score: 0.462, acc: 0.743
[18,1974], f1_score: 0.457, acc: 0.743
[18,2230], f1_score: 0.463, acc: 0.744
[18,2486], f1_score: 0.466, acc:

...

[6,1938], f1_score: 0.450, acc: 0.741
[6,2194], f1_score: 0.449, acc: 0.742
[6,2450], f1_score: 0.451, acc: 0.742
[6,2706], f1_score: 0.461, acc: 0.744
[7,85], f1_score: 0.459, acc: 0.744
[7,341], f1_score: 0.476, acc: 0.748
[7,597], f1_score: 0.479, acc: 0.747
[7,853], f1_score: 0.480, acc: 0.749
[7,1109], f1_score: 0.481, acc: 0.749
[7,1365], f1_score: 0.488, acc: 0.751
[7,1621], f1_score: 0.487, acc: 0.751
[7,1877], f1_score: 0.493, acc: 0.753
[7,2133], f1_score: 0.500, acc: 0.754
[7,2389], f1_score: 0.506, acc: 0.755
[7,2645], f1_score: 0.513, acc: 0.756
[8,24], f1_score: 0.514, acc: 0.757
[8,280], f1_score: 0.516, acc: 0.758
[8,536], f1_score: 0.524, acc: 0.759
[8,792], f1_score: 0.522, acc: 0.759
[8,1048], f1_score: 0.523, acc: 0.760
[8,1304], f1_score: 0.532, acc: 0.761
[8,1560], f1_score: 0.533, acc: 0.762
[8,1816], f1_score: 0.531, acc: 0.762
[8,2072], f1_score: 0.533, acc: 0.762
[8,2328], f1_score: 0.539, acc: 0.764
[8,2584], f1_score: 0.543, acc: 0.764
[8,2840], f1_score: 0.

[25,2059], f1_score: 0.623, acc: 0.788
[25,2315], f1_score: 0.618, acc: 0.788
[25,2571], f1_score: 0.617, acc: 0.788
[25,2827], f1_score: 0.618, acc: 0.788
[26,206], f1_score: 0.619, acc: 0.788
[26,462], f1_score: 0.619, acc: 0.788
[26,718], f1_score: 0.619, acc: 0.788
[26,974], f1_score: 0.618, acc: 0.788
[26,1230], f1_score: 0.619, acc: 0.788
[26,1486], f1_score: 0.620, acc: 0.788
[26,1742], f1_score: 0.623, acc: 0.788
[26,1998], f1_score: 0.623, acc: 0.788
[26,2254], f1_score: 0.619, acc: 0.788
[26,2510], f1_score: 0.620, acc: 0.788
[26,2766], f1_score: 0.621, acc: 0.789
[27,145], f1_score: 0.621, acc: 0.789
[27,401], f1_score: 0.620, acc: 0.789
[27,657], f1_score: 0.618, acc: 0.788
[27,913], f1_score: 0.618, acc: 0.788
[27,1169], f1_score: 0.621, acc: 0.789
[27,1425], f1_score: 0.623, acc: 0.789
[27,1681], f1_score: 0.621, acc: 0.789
[27,1937], f1_score: 0.622, acc: 0.789
[27,2193], f1_score: 0.623, acc: 0.788
[27,2449], f1_score: 0.620, acc: 0.788
[27,2705], f1_score: 0.621, acc: 

[1,1731], f1_score: 0.332, acc: 0.685
[1,1987], f1_score: 0.330, acc: 0.687
[1,2243], f1_score: 0.331, acc: 0.689
[1,2499], f1_score: 0.338, acc: 0.690
[1,2755], f1_score: 0.346, acc: 0.692
[2,134], f1_score: 0.343, acc: 0.694
[2,390], f1_score: 0.341, acc: 0.695
[2,646], f1_score: 0.334, acc: 0.696
[2,902], f1_score: 0.328, acc: 0.699
[2,1158], f1_score: 0.328, acc: 0.699
[2,1414], f1_score: 0.334, acc: 0.700
[2,1670], f1_score: 0.329, acc: 0.700
[2,1926], f1_score: 0.335, acc: 0.700
[2,2182], f1_score: 0.331, acc: 0.701
[2,2438], f1_score: 0.343, acc: 0.701
[2,2694], f1_score: 0.360, acc: 0.703
[3,73], f1_score: 0.360, acc: 0.704
[3,329], f1_score: 0.363, acc: 0.706
[3,585], f1_score: 0.359, acc: 0.706
[3,841], f1_score: 0.361, acc: 0.707
[3,1097], f1_score: 0.365, acc: 0.709
[3,1353], f1_score: 0.366, acc: 0.708
[3,1609], f1_score: 0.360, acc: 0.709
[3,1865], f1_score: 0.360, acc: 0.709
[3,2121], f1_score: 0.362, acc: 0.710
[3,2377], f1_score: 0.369, acc: 0.711
[3,2633], f1_score: 0

[20,2108], f1_score: 0.597, acc: 0.781
[20,2364], f1_score: 0.595, acc: 0.781
[20,2620], f1_score: 0.599, acc: 0.781
[20,2876], f1_score: 0.599, acc: 0.781
[21,255], f1_score: 0.600, acc: 0.781
[21,511], f1_score: 0.600, acc: 0.781
[21,767], f1_score: 0.599, acc: 0.782
[21,1023], f1_score: 0.597, acc: 0.781
[21,1279], f1_score: 0.599, acc: 0.781
[21,1535], f1_score: 0.603, acc: 0.782
[21,1791], f1_score: 0.603, acc: 0.782
[21,2047], f1_score: 0.602, acc: 0.782
[21,2303], f1_score: 0.603, acc: 0.783
[21,2559], f1_score: 0.602, acc: 0.782
[21,2815], f1_score: 0.601, acc: 0.782
[22,194], f1_score: 0.603, acc: 0.782
[22,450], f1_score: 0.603, acc: 0.783
[22,706], f1_score: 0.604, acc: 0.783
[22,962], f1_score: 0.601, acc: 0.782
[22,1218], f1_score: 0.604, acc: 0.783
[22,1474], f1_score: 0.604, acc: 0.783
[22,1730], f1_score: 0.603, acc: 0.783
[22,1986], f1_score: 0.603, acc: 0.783
[22,2242], f1_score: 0.603, acc: 0.782
[22,2498], f1_score: 0.604, acc: 0.782
[22,2754], f1_score: 0.605, acc:

[39,1973], f1_score: 0.624, acc: 0.789
[39,2229], f1_score: 0.622, acc: 0.789
[39,2485], f1_score: 0.620, acc: 0.789
[39,2741], f1_score: 0.621, acc: 0.789
[40,120], f1_score: 0.624, acc: 0.790
[40,376], f1_score: 0.621, acc: 0.789
[40,632], f1_score: 0.623, acc: 0.790
[40,888], f1_score: 0.621, acc: 0.789
[40,1144], f1_score: 0.625, acc: 0.791
[40,1400], f1_score: 0.622, acc: 0.789
[40,1656], f1_score: 0.621, acc: 0.789
[40,1912], f1_score: 0.625, acc: 0.790
[40,2168], f1_score: 0.626, acc: 0.789
[40,2424], f1_score: 0.622, acc: 0.789
[40,2680], f1_score: 0.622, acc: 0.789
[41,59], f1_score: 0.624, acc: 0.789
[41,315], f1_score: 0.623, acc: 0.790
[41,571], f1_score: 0.623, acc: 0.790
[41,827], f1_score: 0.622, acc: 0.791
[41,1083], f1_score: 0.623, acc: 0.790
[41,1339], f1_score: 0.626, acc: 0.790
[41,1595], f1_score: 0.624, acc: 0.789
[41,1851], f1_score: 0.624, acc: 0.790
[41,2107], f1_score: 0.626, acc: 0.790
[41,2363], f1_score: 0.623, acc: 0.789
[41,2619], f1_score: 0.625, acc: 0

In [84]:
max_score = 0
max_model_id_lst = []
for i in model_eval_dict:
    if model_eval_dict[i]['f1score'] > max_score:
        max_model_id_lst = []
        max_model_id_lst.append(i)
        max_score = model_eval_dict[i]['f1score']
    elif model_eval_dict[i]['f1score'] == max_score:
        max_model_id_lst.append(i)

In [85]:
max_model_id_lst

[58]

In [86]:
model_eval_dict[58] # GIT: prec=0.702, recall=0.565, F1=0.626, acc=0.788

{'precision': 0.7101209151478831,
 'recall': 0.5670780303081782,
 'f1score': 0.6305398924429865,
 'accuracy': 0.7908811569636365}

In [87]:
hyper_param_dict[58]

{'hidden_size': 512,
 'attention_size': 200,
 'attention_head': 128,
 'learning_rate': 0.0003,
 'max_iter': 122880,
 'max_fscore': 0.7,
 'batch_size': 16,
 'test_batch_size': 512,
 'test_inc_size': 256,
 'dropout_rate': 0.5,
 'weight_decay': 1e-05,
 'initializtion': False,
 'attention': True,
 'cancer_type': True,
 'deg_shuffle': False}

In [89]:
params_dict = hyper_param_dict[58]
args.hidden_size = params_dict["hidden_size"]
args.attention_size=params_dict["attention_size"] 
args.attention_head=params_dict["attention_head"] 
args.learning_rate=params_dict["learning_rate"] 
args.max_iter=params_dict["max_iter"]
args.max_fscore=params_dict["max_fscore"] 
args.batch_size=params_dict["batch_size"] 
args.test_batch_size=params_dict["test_batch_size"]
args.test_inc_size=params_dict["test_inc_size"]
args.dropout_rate=params_dict["dropout_rate"]
args.weight_decay=params_dict["weight_decay"] 
args.initializtion=params_dict["initializtion"] 
args.attention=params_dict["attention"]
args.cancer_type=params_dict["cancer_type"] 
args.deg_shuffle=params_dict["deg_shuffle"]
model = GIT(args)   # initialize GIT model (with no pretrained gene embeddings)
model.build()  

model.load_model(os.path.join(f"ht_models/init_trained_model_58.pth"))
model.layer_sga_emb.weight.detach().numpy().shape

Loading model from ht_models/init_trained_model_58.pth


(324, 512)

In [90]:
gene_embeddings_324 = model.layer_sga_emb.weight.detach().numpy()
with open('gene_emb_324_ht.npy', 'wb') as f: ### gene embeddings trained without pretrained embeddings
    np.save(f, gene_embeddings_324)

In [91]:
### Load the pretrained gene-embeddings
gene_emb_pretrain_full = np.load('gene_emb_pretrain_5_epoch.npy')

with open('idx2gene_5_epoch.txt') as f:
    idx2gene = f.readlines()[0]
    
idx = list(map(lambda x: int(x.split(': ')[0]), idx2gene.split(', ')))
genes = list(map(lambda x: x.split(': ')[1][1:-1], idx2gene.split(', ')))

idx2gene = {}
gene2idx = {}
for i in range(len(idx)):
    gene = genes[i]
    idx2gene[idx[i]] = gene
    gene2idx[gene] = idx[i]

In [92]:
import json
f = open('new_idx2sga.txt')
idx2sga = json.load(f)

In [93]:
gene_embs = [[0]*512]
helper_dict = {'FAM123B': 'AMER1', 'FAM46C': 'TENT5C', 'C17orf39': 'GID4', 'H3F3A': 'H3-3A', 'MLL': 'KMT2A',
            'MLL2': 'KMT2D', 'MRE11A': 'MRE11', 'MYCL1': 'MYCL', 'PARK2': 'PRKN', 'WHSC1': 'NSD2', 'WHSC1L1': 'NSD3'}
not_found = []
for i in range(len(idx2sga.keys())):
    gene = idx2sga[str(i)]
    try:
        if gene in gene2idx:
            new_idx = gene2idx[gene]
        else:
            new_idx = gene2idx[helper_dict[gene]]
    except:
        not_found.append(gene)
    gene_embs.append(gene_emb_pretrain_full[new_idx])

In [94]:
gene_emb_pretrain = np.array(gene_embs)
gene_emb_pretrain.shape

(324, 512)

In [95]:
with open('gene_emb_pretrain_324.npy', 'wb') as f:
    np.save(f, gene_emb_pretrain)

In [96]:
print("Loading dataset...")
dataset = load_dataset(input_dir='', deg_shuffle=args.deg_shuffle)
train_set, test_set = split_dataset(dataset, ratio=0.66)

args.can_size = dataset["can"].max()        # cancer type dimension
args.sga_size = dataset["sga"].max()        # SGA dimension
args.deg_size = dataset["deg"].shape[1]     # DEG output dimension
args.num_max_sga = dataset["sga"].shape[1]  # maximum number of SGAs in a tumor

Loading dataset...


In [97]:
### Hyperparameters set

params_lst = ["hidden_size", "attention_size", "attention_head", "learning_rate", "max_iter", "max_fscore", "batch_size",
             "test_batch_size", "test_inc_size", "dropout_rate", "weight_decay", "initializtion", "attention",
             "cancer_type", "deg_shuffle"]
new_params_value_dict = {"hidden_size" : [256, 512], "attention_size":[200, 400], 
                     "attention_head": [128, 256], "learning_rate": [1e-4, 2e-4, 3e-4], 
                     "max_iter": [3072*20, 3072*40], "max_fscore": [0.7, 0.8], 
                     "batch_size": [16], "test_batch_size": [512], 
                     "test_inc_size": [256], "dropout_rate": [0.5], 
                     "weight_decay": [1e-5], "initializtion": [True], 
                     "attention": [True], "cancer_type": [True], "deg_shuffle": [False]}
import itertools as it

new_allparams = new_params_value_dict
new_combinations = it.product(*(new_params_value_dict[param] for param in new_allparams))
new_combinations_lst = list(new_combinations)
print(len(new_combinations_lst))

new_hyper_param_dict = {}
for i in range(len(new_combinations_lst)):
    new_hyper_param_dict[i] = {}
    for j in range(len(params_lst)):
        new_hyper_param_dict[i][params_lst[j]] = new_combinations_lst[i][j]

96


In [98]:
import numpy
print("Original Hyperparameters:")
print(args)

Original Hyperparameters:
Namespace(f='C:\\Users\\Shi Yingfei\\AppData\\Roaming\\jupyter\\runtime\\kernel-639b48bb-9bbd-41e6-83b7-070f3f769436.json', train_model=True, input_dir='data', output_dir='data', embedding_size=512, hidden_size=512, attention_size=200, attention_head=128, learning_rate=0.0003, max_iter=122880, max_fscore=0.7, batch_size=16, test_batch_size=512, test_inc_size=256, dropout_rate=0.5, weight_decay=1e-05, initializtion=False, attention=True, cancer_type=True, deg_shuffle=False, can_size=16, sga_size=323, deg_size=2207, num_max_sga=41)


In [99]:
new_model_eval_dict = {}
for i in hyper_param_dict:
    params_dict = new_hyper_param_dict[i]
    args.hidden_size = params_dict["hidden_size"]
    args.attention_size=params_dict["attention_size"] 
    args.attention_head=params_dict["attention_head"] 
    args.learning_rate=params_dict["learning_rate"] 
    args.max_iter=params_dict["max_iter"]
    args.max_fscore=params_dict["max_fscore"] 
    args.batch_size=params_dict["batch_size"] 
    args.test_batch_size=params_dict["test_batch_size"]
    args.test_inc_size=params_dict["test_inc_size"]
    args.dropout_rate=params_dict["dropout_rate"]
    args.weight_decay=params_dict["weight_decay"] 
    args.initializtion=params_dict["initializtion"] 
    args.attention=params_dict["attention"]
    args.cancer_type=params_dict["cancer_type"] 
    args.deg_shuffle=params_dict["deg_shuffle"]
    model = GIT(args)   # initialize GIT model (with pretrained gene embeddings)
    model.build()                               # build GIT model

    if args.train_model:                        # train from scratch
        print(f"Model_{i}_Training...")
        model.train(train_set, test_set,
            batch_size=args.batch_size,
            test_batch_size=args.test_batch_size,
            max_iter=args.max_iter,
            max_fscore=args.max_fscore,
            test_inc_size=args.test_inc_size)
        model.save_model(os.path.join(f"ht_models/pret_trained_model_{i}.pth"))
    else:                                      # or directly load trained model
        model.load_model(os.path.join(f"ht_models/pret_trained_model_{i}.pth"))
        
    # evaluation
    print("Evaluating...")
    labels, preds, _, _, _, _, _ = model.test(test_set, test_batch_size=args.test_batch_size)
    precision, recall, f1score, accuracy = evaluate(labels, preds, epsilon=model.epsilon)
    print("prec=%.3f, recall=%.3f, F1=%.3f, acc=%.3f"%(precision, recall, f1score, accuracy))
    
    new_model_eval_dict[i] = {"precision":precision, "recall":recall, "f1score":f1score, "accuracy":accuracy}

Model_0_Training...


C:\Users\Shi Yingfei\AppData\Local\Temp\ipykernel_15320\910214608.py:112: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  A = F.softmax(E_t2)


[0,0], f1_score: 0.389, acc: 0.504
[0,256], f1_score: 0.386, acc: 0.523
[0,512], f1_score: 0.385, acc: 0.535
[0,768], f1_score: 0.384, acc: 0.542
[0,1024], f1_score: 0.383, acc: 0.547
[0,1280], f1_score: 0.383, acc: 0.552
[0,1536], f1_score: 0.382, acc: 0.557
[0,1792], f1_score: 0.382, acc: 0.562
[0,2048], f1_score: 0.380, acc: 0.567
[0,2304], f1_score: 0.381, acc: 0.575
[0,2560], f1_score: 0.379, acc: 0.582
[0,2816], f1_score: 0.378, acc: 0.592
[1,195], f1_score: 0.377, acc: 0.601
[1,451], f1_score: 0.376, acc: 0.610
[1,707], f1_score: 0.374, acc: 0.619
[1,963], f1_score: 0.371, acc: 0.627
[1,1219], f1_score: 0.371, acc: 0.635
[1,1475], f1_score: 0.370, acc: 0.640
[1,1731], f1_score: 0.368, acc: 0.648
[1,1987], f1_score: 0.366, acc: 0.651
[1,2243], f1_score: 0.366, acc: 0.655
[1,2499], f1_score: 0.365, acc: 0.659
[1,2755], f1_score: 0.366, acc: 0.661
[2,134], f1_score: 0.366, acc: 0.662
[2,390], f1_score: 0.365, acc: 0.665
[2,646], f1_score: 0.364, acc: 0.669
[2,902], f1_score: 0.362,

[19,633], f1_score: 0.499, acc: 0.750
[19,889], f1_score: 0.498, acc: 0.750
[19,1145], f1_score: 0.499, acc: 0.751
[19,1401], f1_score: 0.499, acc: 0.751
[19,1657], f1_score: 0.498, acc: 0.750
[19,1913], f1_score: 0.497, acc: 0.751
[19,2169], f1_score: 0.500, acc: 0.752
[19,2425], f1_score: 0.501, acc: 0.752
[19,2681], f1_score: 0.505, acc: 0.752
[20,60], f1_score: 0.502, acc: 0.751
[20,316], f1_score: 0.515, acc: 0.754
[20,572], f1_score: 0.512, acc: 0.754
[20,828], f1_score: 0.512, acc: 0.754
[20,1084], f1_score: 0.512, acc: 0.754
[20,1340], f1_score: 0.512, acc: 0.754
[20,1596], f1_score: 0.516, acc: 0.755
[20,1852], f1_score: 0.517, acc: 0.755
[20,2108], f1_score: 0.518, acc: 0.756
[20,2364], f1_score: 0.521, acc: 0.757
[20,2620], f1_score: 0.519, acc: 0.755
[20,2876], f1_score: 0.523, acc: 0.757
[21,255], f1_score: 0.525, acc: 0.757
[21,511], f1_score: 0.526, acc: 0.758
[21,767], f1_score: 0.524, acc: 0.758
[21,1023], f1_score: 0.524, acc: 0.758
Saving model to ht_models/pret_trai

[16,1840], f1_score: 0.466, acc: 0.742
[16,2096], f1_score: 0.468, acc: 0.742
[16,2352], f1_score: 0.467, acc: 0.742
[16,2608], f1_score: 0.474, acc: 0.742
[16,2864], f1_score: 0.470, acc: 0.743
[17,243], f1_score: 0.474, acc: 0.743
[17,499], f1_score: 0.473, acc: 0.744
[17,755], f1_score: 0.476, acc: 0.744
[17,1011], f1_score: 0.469, acc: 0.744
[17,1267], f1_score: 0.472, acc: 0.744
[17,1523], f1_score: 0.473, acc: 0.745
[17,1779], f1_score: 0.476, acc: 0.746
[17,2035], f1_score: 0.483, acc: 0.746
[17,2291], f1_score: 0.478, acc: 0.746
[17,2547], f1_score: 0.484, acc: 0.746
[17,2803], f1_score: 0.488, acc: 0.747
[18,182], f1_score: 0.492, acc: 0.748
[18,438], f1_score: 0.491, acc: 0.748
[18,694], f1_score: 0.492, acc: 0.748
[18,950], f1_score: 0.485, acc: 0.747
[18,1206], f1_score: 0.491, acc: 0.748
[18,1462], f1_score: 0.494, acc: 0.749
[18,1718], f1_score: 0.491, acc: 0.749
[18,1974], f1_score: 0.493, acc: 0.750
[18,2230], f1_score: 0.496, acc: 0.751
[18,2486], f1_score: 0.498, acc:

...

[6,1938], f1_score: 0.403, acc: 0.722
[6,2194], f1_score: 0.405, acc: 0.723
[6,2450], f1_score: 0.406, acc: 0.723
[6,2706], f1_score: 0.420, acc: 0.725
[7,85], f1_score: 0.417, acc: 0.721
[7,341], f1_score: 0.411, acc: 0.722
[7,597], f1_score: 0.415, acc: 0.724
[7,853], f1_score: 0.407, acc: 0.724
[7,1109], f1_score: 0.417, acc: 0.725
[7,1365], f1_score: 0.419, acc: 0.726
[7,1621], f1_score: 0.413, acc: 0.726
[7,1877], f1_score: 0.411, acc: 0.726
[7,2133], f1_score: 0.412, acc: 0.727
[7,2389], f1_score: 0.417, acc: 0.728
[7,2645], f1_score: 0.433, acc: 0.728
[8,24], f1_score: 0.426, acc: 0.729
[8,280], f1_score: 0.427, acc: 0.727
[8,536], f1_score: 0.435, acc: 0.728
[8,792], f1_score: 0.427, acc: 0.730
[8,1048], f1_score: 0.430, acc: 0.729
[8,1304], f1_score: 0.443, acc: 0.731
[8,1560], f1_score: 0.428, acc: 0.729
[8,1816], f1_score: 0.423, acc: 0.729
[8,2072], f1_score: 0.434, acc: 0.731
[8,2328], f1_score: 0.431, acc: 0.732
[8,2584], f1_score: 0.445, acc: 0.733
[8,2840], f1_score: 0.

[25,2059], f1_score: 0.620, acc: 0.787
[25,2315], f1_score: 0.619, acc: 0.787
[25,2571], f1_score: 0.617, acc: 0.787
[25,2827], f1_score: 0.620, acc: 0.788
[26,206], f1_score: 0.620, acc: 0.788
[26,462], f1_score: 0.618, acc: 0.787
[26,718], f1_score: 0.616, acc: 0.788
[26,974], f1_score: 0.617, acc: 0.788
[26,1230], f1_score: 0.620, acc: 0.788
[26,1486], f1_score: 0.618, acc: 0.788
[26,1742], f1_score: 0.621, acc: 0.788
[26,1998], f1_score: 0.620, acc: 0.788
[26,2254], f1_score: 0.619, acc: 0.787
[26,2510], f1_score: 0.614, acc: 0.787
[26,2766], f1_score: 0.618, acc: 0.787
[27,145], f1_score: 0.618, acc: 0.788
[27,401], f1_score: 0.617, acc: 0.788
[27,657], f1_score: 0.619, acc: 0.788
[27,913], f1_score: 0.618, acc: 0.787
[27,1169], f1_score: 0.619, acc: 0.788
[27,1425], f1_score: 0.622, acc: 0.788
[27,1681], f1_score: 0.621, acc: 0.788
[27,1937], f1_score: 0.622, acc: 0.788
[27,2193], f1_score: 0.611, acc: 0.786
[27,2449], f1_score: 0.622, acc: 0.788
[27,2705], f1_score: 0.617, acc: 

[1,1731], f1_score: 0.344, acc: 0.687
[1,1987], f1_score: 0.346, acc: 0.686
[1,2243], f1_score: 0.349, acc: 0.688
[1,2499], f1_score: 0.352, acc: 0.692
[1,2755], f1_score: 0.358, acc: 0.690
[2,134], f1_score: 0.356, acc: 0.691
[2,390], f1_score: 0.352, acc: 0.694
[2,646], f1_score: 0.346, acc: 0.696
[2,902], f1_score: 0.348, acc: 0.696
[2,1158], f1_score: 0.348, acc: 0.696
[2,1414], f1_score: 0.348, acc: 0.697
[2,1670], f1_score: 0.342, acc: 0.698
[2,1926], f1_score: 0.341, acc: 0.697
[2,2182], f1_score: 0.343, acc: 0.700
[2,2438], f1_score: 0.345, acc: 0.700
[2,2694], f1_score: 0.349, acc: 0.700
[3,73], f1_score: 0.349, acc: 0.701
[3,329], f1_score: 0.355, acc: 0.701
[3,585], f1_score: 0.355, acc: 0.701
[3,841], f1_score: 0.358, acc: 0.704
[3,1097], f1_score: 0.344, acc: 0.703
[3,1353], f1_score: 0.361, acc: 0.703
[3,1609], f1_score: 0.344, acc: 0.705
[3,1865], f1_score: 0.349, acc: 0.704
[3,2121], f1_score: 0.341, acc: 0.705
[3,2377], f1_score: 0.352, acc: 0.706
[3,2633], f1_score: 0

[20,2108], f1_score: 0.616, acc: 0.786
[20,2364], f1_score: 0.613, acc: 0.786
[20,2620], f1_score: 0.616, acc: 0.786
[20,2876], f1_score: 0.616, acc: 0.786
[21,255], f1_score: 0.612, acc: 0.786
[21,511], f1_score: 0.614, acc: 0.786
[21,767], f1_score: 0.610, acc: 0.786
[21,1023], f1_score: 0.614, acc: 0.786
[21,1279], f1_score: 0.609, acc: 0.786
[21,1535], f1_score: 0.618, acc: 0.786
[21,1791], f1_score: 0.615, acc: 0.786
[21,2047], f1_score: 0.618, acc: 0.787
[21,2303], f1_score: 0.616, acc: 0.786
[21,2559], f1_score: 0.612, acc: 0.786
[21,2815], f1_score: 0.615, acc: 0.786
[22,194], f1_score: 0.613, acc: 0.786
[22,450], f1_score: 0.614, acc: 0.786
[22,706], f1_score: 0.613, acc: 0.787
[22,962], f1_score: 0.612, acc: 0.786
[22,1218], f1_score: 0.615, acc: 0.787
[22,1474], f1_score: 0.611, acc: 0.786
[22,1730], f1_score: 0.613, acc: 0.786
[22,1986], f1_score: 0.617, acc: 0.787
[22,2242], f1_score: 0.618, acc: 0.787
[22,2498], f1_score: 0.615, acc: 0.787
[22,2754], f1_score: 0.615, acc:

[39,1973], f1_score: 0.622, acc: 0.789
[39,2229], f1_score: 0.620, acc: 0.787
[39,2485], f1_score: 0.621, acc: 0.788
[39,2741], f1_score: 0.624, acc: 0.789
[40,120], f1_score: 0.620, acc: 0.788
[40,376], f1_score: 0.620, acc: 0.788
[40,632], f1_score: 0.621, acc: 0.788
[40,888], f1_score: 0.621, acc: 0.789
[40,1144], f1_score: 0.623, acc: 0.789
[40,1400], f1_score: 0.623, acc: 0.789
[40,1656], f1_score: 0.625, acc: 0.789
[40,1912], f1_score: 0.621, acc: 0.788
[40,2168], f1_score: 0.625, acc: 0.789
[40,2424], f1_score: 0.623, acc: 0.789
[40,2680], f1_score: 0.622, acc: 0.788
[41,59], f1_score: 0.626, acc: 0.789
[41,315], f1_score: 0.622, acc: 0.789
[41,571], f1_score: 0.621, acc: 0.789
[41,827], f1_score: 0.620, acc: 0.788
[41,1083], f1_score: 0.624, acc: 0.789
[41,1339], f1_score: 0.620, acc: 0.789
[41,1595], f1_score: 0.620, acc: 0.789
[41,1851], f1_score: 0.623, acc: 0.789
[41,2107], f1_score: 0.625, acc: 0.789
[41,2363], f1_score: 0.620, acc: 0.789
[41,2619], f1_score: 0.625, acc: 0

In [100]:
max_score = 0
max_model_id_lst = []
for i in new_model_eval_dict:
    if new_model_eval_dict[i]['f1score'] > max_score:
        max_model_id_lst = []
        max_model_id_lst.append(i)
        max_score = new_model_eval_dict[i]['f1score']
    elif new_model_eval_dict[i]['f1score'] == max_score:
        max_model_id_lst.append(i)

In [101]:
max_model_id_lst

[59]

In [103]:
new_model_eval_dict[59] # GIT: prec=0.702, recall=0.565, F1=0.626, acc=0.788

{'precision': 0.7033453534082631,
 'recall': 0.5670692937249006,
 'f1score': 0.6278488070985789,
 'accuracy': 0.7884582892476308}

In [102]:
new_hyper_param_dict[59]

{'hidden_size': 512,
 'attention_size': 200,
 'attention_head': 128,
 'learning_rate': 0.0003,
 'max_iter': 122880,
 'max_fscore': 0.8,
 'batch_size': 16,
 'test_batch_size': 512,
 'test_inc_size': 256,
 'dropout_rate': 0.5,
 'weight_decay': 1e-05,
 'initializtion': True,
 'attention': True,
 'cancer_type': True,
 'deg_shuffle': False}

In [104]:
params_dict = new_hyper_param_dict[59]
args.hidden_size = params_dict["hidden_size"]
args.attention_size=params_dict["attention_size"] 
args.attention_head=params_dict["attention_head"] 
args.learning_rate=params_dict["learning_rate"] 
args.max_iter=params_dict["max_iter"]
args.max_fscore=params_dict["max_fscore"] 
args.batch_size=params_dict["batch_size"] 
args.test_batch_size=params_dict["test_batch_size"]
args.test_inc_size=params_dict["test_inc_size"]
args.dropout_rate=params_dict["dropout_rate"]
args.weight_decay=params_dict["weight_decay"] 
args.initializtion=params_dict["initializtion"] 
args.attention=params_dict["attention"]
args.cancer_type=params_dict["cancer_type"] 
args.deg_shuffle=params_dict["deg_shuffle"]
model = GIT(args)   # initialize GIT model (with pretrained gene embeddings)
model.build()  

model.load_model(os.path.join(f"ht_models/pret_trained_model_59.pth"))
model.layer_sga_emb.weight.detach().numpy().shape

Loading model from ht_models/pret_trained_model_59.pth


(324, 512)

In [105]:
gene_embeddings_finetune = model.layer_sga_emb.weight.detach().numpy()

In [106]:
with open('gene_emb_finetune_324_ht.npy', 'wb') as f: ### gene embeddings trained with pretrained embeddings (5-epoch)
    np.save(f, gene_embeddings_finetune)